In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import time

standings_url= 'https://fbref.com/en/comps/9/Premier-League-Stats'
#standings_url

data = requests.get(standings_url)
#data
#data.text

soup = BeautifulSoup(data.text)
#soup



# getting table with matches and links
standings_table=soup.select('table.stats_table')[0]
#standings_table

In [ ]:
links = standings_table.find_all('a')
#print(links)

# loop through 'a' elements and find 'href'
links = [l.get('href') for l in links]


# loop through links selected and get only team names, checking if contains substring in href string
links = [l for l in links if '/squads/' in l]
links

In [ ]:
# add string for each link for complete url
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

In [ ]:
team_url = team_urls[0]

data = requests.get(team_url)
#data.text

In [ ]:

# read all tables on page and look for substring inside the string
matches = pd.read_html(data.text, match="Scores & Fixtures")
matches[0].head()

In [ ]:

# getting SHOOTING data
soup= BeautifulSoup(data.text)
links=soup.find_all('a')

links = [l.get('href') for l in links]

links = [l for l in links if l and 'all_comps/shooting/' in l]

links

In [ ]:
# add url format to desired link
data = requests.get(f"https://fbref.com{links[0]}")
#data.text

shooting = pd.read_html(data.text, match="Shooting")[0]
#shooting.head()

# drop one header
shooting.columns = shooting.columns.droplevel()
shooting.head()

In [ ]:
# combining two tables on Date information
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
#team_data.head()


In [ ]:
# list of years


In [ ]:

years = list(range(2022, 2020, -1))

#years 

all_matches = []

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


# loop through years; combining code

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    links = [l.get('href') for l in standings_table.find_all('a')]
    links = [l for l in links if l and '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    # getting previous season link for scraping
    previous_season = soup.select('a.prev')[0].get('href')
    standings_url = f'https://fbref.com{previous_season}'
    
    # splicing data, keeping only team name
    for team_urls in team_urls:
        team_name = team_url.split('/')[-1].replace('-Stats', '').replace('-',' ')
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        soup = BeautifulSoup(data.text)
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
            
        team_data = team_data[team_data["Comp"]=='Premier League']
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)


    
    
#team_urls
#team_url
#team_url.split('/')[-1].replace('-Stats', '').replace('-',' ')

In [ ]:
# agregating dataframes in one
match_df = pd.concat(all_matches)

# lowering characters, just in case
match_df.columns = [c.lower() for c in match_df.columns]
match_df

match_df.to_csv("matches.csv")